###  Revisiting the Evolution of Bow-tie Architecture in Signaling Networks: Non linear simulation

In [1]:
import sys
import numpy as np
import copy
import matplotlib.pyplot as plt
import random
import math
np.set_printoptions(precision=4, floatmode='maxprec')
np.set_printoptions(suppress=True)
from graphviz import Graph
from graphviz import Digraph
import os 
from scipy.stats import special_ortho_group


class genom:

    genom_list = None
    evaluation = None
    input_layer = None
    Thr = None

    def __init__(self, input_layer, genom_list, evaluation, Thr):
        self.genom_list = genom_list
        self.evaluation = evaluation
        self.input_layer = input_layer
        self.Thr = Thr


    def getGenom(self):
        return self.genom_list
    
    def getInputLayer(self):
        return self.input_layer
    
    def getThr(self):
        return self.Thr


    def getEvaluation(self):
        return self.evaluation


    def setGenom(self, genom_list):
        self.genom_list = genom_list
    
    def setInputLayer(self, input_layer):
        self.input_layer = input_layer
        
    def setThr(self, Thr):
        self.Thr = Thr


    def setEvaluation(self, evaluation):
        self.evaluation = evaluation


def WriteNetwork(Ind, nNode, nLayer, count, tidyvisual, correction_v, aling_center):
    #dg = Digraph(format='png')
    dg = Digraph(format='png')
    if tidyvisual:
        dg.attr('node', shape='circle', label='')
    else:
        dg.attr('node', shape='circle')
        
    active_node_arr = Active_node_in_MATLAB(Ind, "test")
    #Arrange Nodes
    matrices = Ind.getGenom()
    for layer in range(nLayer):
        for node in range(nNode):
            width = active_node_arr[layer][node]
            name = str(layer)+str(node)
            
            if tidyvisual:
                if width:
                    dg.node(name, penwidth=str(width), style='filled', fillcolor='skyblue')
                elif not aling_center:
                    dg.node(name, penwidth=str(width))
            else:
                dg.node(name, penwidth=str(width))
            
    dg.attr(label="Generation:" + str(count) + "\l" + "fitness:" + str(np.round(Ind.getEvaluation(),4)))
    
    #if count%1000 <= 200:
    #    label_ = "Goal matrix change"
    #else:
    #    label_ = "     "
    label_ = "     "
    
    dg.attr(label="Fitness: " + str(np.round(Ind.getEvaluation(),2)) + 
            "   Generation: " + str(count) + "\n" + label_)
    #dg.node("test", label="Goal matrix change", penwidth=0.1)
    dg.attr(fontsize='16')
    #Write links
    for layer in range(1, nLayer):
        for postnode in range(nNode):
            #row
            post = str(layer) + str(postnode)
            for prenode in range(nNode):
                #column
                pre= str(layer-1) + str(prenode)
                intensity =  matrices[layer-1][postnode, prenode]
                if intensity < 0:
                    color_ = "blue"
                    intensity = abs(intensity)
                else:
                    color_ = "black"
                    
                if tidyvisual:
                    if active_node_arr[layer][postnode] & active_node_arr[layer-1][prenode]:
                        if intensity > correction_v:
                            intensity = correction_v
                        dg.edge(pre, post,  penwidth=str(intensity), arrowsize="0.5", color=color_)
                    elif not aling_center:
                        dg.edge(pre, post,  penwidth='0', arrowsize="0", color=color_)
                else:
                    dg.edge(pre, post,  penwidth=str(intensity), arrowsize="0", color=color_)
    return(dg)



In [2]:

"""
Define functions required for evolutionary simulation
"""


def CreateInd(nNode, nMatrix, norm):
    #This function returns Node x Node x Layer Matrix, which describe individual's network structure
    pre_input_layer = np.random.uniform(-1, 1, (nNode, 64))
    #pre_input_layer = np.random.uniform(0, 0.05, (nNode, 64))
    #pre_net =  np.random.uniform(0, 0.05, (nMatrix, nNode, nNode))
    pre_net =  np.random.uniform(-1, 1, (nMatrix, nNode, nNode))
    out = np.dot(Total_in_out(pre_net), pre_input_layer)
    pre_net_norm = (np.linalg.norm(out, ord="fro")) 
    normalizeF = (norm/pre_net_norm)**(1/(nMatrix+1))
    network = normalizeF*pre_net
    input_layer = normalizeF*pre_input_layer
    
    #Thr = np.random.uniform(-1, 1, (nNode,nMatrix+1))
    Thr = np.zeros((nNode,nMatrix+1))
    
    return genom(input_layer, network, 0, Thr)

# 64 x 10

def Total_in_out(Ind):
    z = np.dot(Ind[nMatrix -1], Ind[nMatrix -2])
    for i in range(nMatrix -3, -1, -1):
        z = np.dot(z, Ind[i])
    return z

"""
def evaluation(input_layer, middle_network):
    middle_in_out = Total_in_out(middle_network)
    inout = np.dot(middle_in_out, input_layer)
    out = np.dot(inout, DIG_INPUT.T) 
    softmax_out = [np.exp(out[:,i])/np.sum(np.exp(out[:,i])) for i in range(out.shape[1])]
    pred = [np.argmax(softmax_out[i]) for i in range(out.shape[1])]
    eva = np.log(accuracy_score(DIG_TRUE, pred))
    return(eva)
#evaluation(IND.getInputLayer(), Total_in_out(IND.getGenom()))
"""

#MUTATION_RATE = 0.2/((nMatrix*nNode*nNode) + 64*nNode)
def add_mutation(Ind, corrected_mut_rate):
    var = 0.5
    #var = 0.1
    bitmask = np.where(np.random.uniform(0, 1, (nMatrix, nNode, nNode))  < corrected_mut_rate, 1, 0)
    noise_matrix = np.random.normal(1, var, (nMatrix, nNode, nNode))
    pre_mult_mat = bitmask*noise_matrix
    #mult_mat = np.where(pre_mult_mat == 0,  1, pre_mult_mat)
    mult_mat = np.where(pre_mult_mat == 0,  1, pre_mult_mat)
    mutated_network = Ind.getGenom()*mult_mat
    Ind.setGenom(mutated_network)
    
    bitmask = np.where(np.random.uniform(0, 1, (nNode, 64))  < corrected_mut_rate, 1, 0)
    noise_matrix = np.random.normal(1, var, (nNode, 64))
    pre_mult_mat = bitmask*noise_matrix
    mult_mat = np.where(pre_mult_mat == 0,  1, pre_mult_mat)
    mutated_input_layer = Ind.getInputLayer()*mult_mat
    Ind.setInputLayer(mutated_input_layer)

    """
    Thr_var = 0.1
    bitmask = np.where(np.random.uniform(0, 1, (nNode, nMatrix+1))  < corrected_mut_rate, 1, 0)
    noise_matrix = np.random.normal(0, Thr_var, (nNode, nMatrix+1))
    pre_mult_mat = bitmask*noise_matrix
    mult_mat = np.where(pre_mult_mat == 0,  1, pre_mult_mat)
    mutated_Thr = Ind.getThr()*mult_mat
    Ind.setThr(mutated_Thr)
    """
    
    
    return Ind

def select(population, tournament_size):    
    #Tournament selection
    tournament_groups = [random.sample(population, tournament_size) for i in range(POPULATION_SIZE)]
    selected = [ sorted(tournament_group, reverse=True, key=lambda u: u.evaluation)[0] for tournament_group in tournament_groups]
    return selected


"""
def select(population, tournament_size):
    #エリート選択 25個体
    upper_ind = sorted(population, reverse=True, key=lambda u: u.evaluation)[0:100]
    #非エリートから75個体選択
    #non_upper_ind = [ind for ind in population if ind not in upper_ind]
    #random_ind = random.sample(non_upper_ind, 75)
    return(upper_ind)
"""
"""
def select(population, tournament_size=None):
    #random selection
    random_ind = random.sample(population, 100)
    return  random_ind
"""

def Deleted_fitness(Ind, node, layer):
    modified_network =  copy.deepcopy(Ind.getGenom())
    modified_input_layer = copy.deepcopy(Ind.getInputLayer())
    
    Input_link_layer = layer - 1
    Output_link_layer = layer
    
    if layer == -1: #Eliminate input layer
        modified_input_layer[:,node] = 0  #eliminate output
    elif layer == 0:
        modified_input_layer[node,] = 0  #eliminate output
        modified_network[0][:,node] = 0
    elif layer == nLayer-1: #Eliminate output layer
        modified_network[Input_link_layer][node,] = 0   #eliminate input
    else: #Eliminate intermidiate layer
        modified_network[Input_link_layer][node,] = 0   #eliminate input from 
        modified_network[Output_link_layer][:,node] = 0
    orig_fit = Ind.getEvaluation()
    modi_fit = evaluation(modified_input_layer, modified_network, Ind.getThr())
    relative_fitness = abs(orig_fit-modi_fit)

    return(relative_fitness)

"""
def Deleted_totalinout(Ind, node, layer):
    modified_network =  copy.deepcopy(Ind)
    Input_link_layer = layer - 1
    Output_link_layer = layer
    if layer == 0: #Eliminate input layer
        modified_network[Output_link_layer][:,node] = 0  #eliminate output
    elif layer == nLayer-1: #Eliminate output layer
        modified_network[Input_link_layer][node,] = 0   #eliminate input
    else: #Eliminate intermidiate layer
        modified_network[Input_link_layer][node,] = 0   #eliminate input from 
        modified_network[Output_link_layer][:,node] = 0
    orig_network = copy.deepcopy(Ind)
    diff = Total_in_out(orig_network) - Total_in_out(modified_network)
    relative_fitness = np.sum(diff**2)  
    return(relative_fitness)
"""

def MaximumInteraction(Ind, node, layer):
    inmax,outmax = 0,0
    if layer > 0:
        inmax = max(Ind.getGenom()[layer-1][node,:])
    if layer < nLayer-1:
        outmax = max(Ind.getGenom()[layer][:,node])
    #threshold: 0.05
    return max(inmax,outmax)


def Relative_fitness_in_layer(layer, mode, Ind):
    if layer == -1:
        relative_fitness = [(Deleted_fitness(Ind, node, layer)) for node in range(64)]
        if np.sum(relative_fitness) != 0:
            relative_fitness_in_layer = np.array(relative_fitness)/np.sum(relative_fitness)
        else:
            relative_fitness_in_layer = np.repeat(1,nNode)
        active_node_test = np.where(relative_fitness_in_layer > 0.001/6,  1, 0)
        #node数による補正をかける
    else:
        relative_fitness = [(Deleted_fitness(Ind, node, layer)) for node in range(nNode)]
        if np.sum(relative_fitness) != 0:
            relative_fitness_in_layer = np.array(relative_fitness)/np.sum(relative_fitness)
        else:
            relative_fitness_in_layer = np.repeat(1,nNode)
        active_node_test = np.where(relative_fitness_in_layer > 0.001,  1, 0)
    #relative_fitness = [MaximumInteraction(Ind, node, layer) for node in range(nNode)]
    #factor = 1 if sum(relative_fitness) == 0 else sum(relative_fitness)
    #print(f"layer:{layer}, relative:{sum(relative_fitness)}")
    #relative_fitness_in_layer = np.array(relative_fitness)/sum(relative_fitness)
    #active_node_test = np.where(relative_fitness_in_layer > 0.001,  1, 0)
    #active_node_test = np.where(relative_fitness_in_layer > 0.05,  1, 0)
    if mode == "result":
        return(sum(active_node_test))
    elif mode == "test":
        return(active_node_test)

def Active_node_in_MATLAB(Ind, mode):
    #orig_fit = abs(evaluation(Ind.getGenom(), DesiredGoal))
    orig_fit = abs(Ind.getEvaluation())
    if orig_fit == 0:
        orig_fit = 0.001
    active_node_list = [Relative_fitness_in_layer(layer, mode, Ind) for layer in range(-1,nLayer)]
    return(active_node_list)


In [4]:
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
digits = load_digits()
# load learning data set
DIG_target = list(digits.target)#[0:300]
tar_0 = [i for i, x in enumerate(DIG_target) if x == 0]
tar_1 = [i for i, x in enumerate(DIG_target) if x == 1]
tar_2 = [i for i, x in enumerate(DIG_target) if x == 2]
tar_3 = [i for i, x in enumerate(DIG_target) if x == 3]
tar_4 = [i for i, x in enumerate(DIG_target) if x == 4]
tar_5 = [i for i, x in enumerate(DIG_target) if x == 5]
tar_6 = [i for i, x in enumerate(DIG_target) if x == 6]
tar_7 = [i for i, x in enumerate(DIG_target) if x == 7]
tar_8 = [i for i, x in enumerate(DIG_target) if x == 8]

tar_index = tar_2 + tar_4 + tar_6 + tar_8

tar_sindex = sorted(tar_index)

DIG_INPUT = digits.data[tar_sindex,]
DIG_TRUE = digits.target[tar_sindex]
print(DIG_INPUT.shape)

DATA_SIZE = len(DIG_TRUE)

#GOAL MATRIX (one hot encoding)



(713, 64)


In [ ]:
def tanh(x):
    y = (np.exp(x)- np.exp(-x)) / (np.exp(x) + np.exp(-x))
    return y


def F(x):
    x_ = copy.deepcopy(x)
    x_ = x_
    x_[np.where(x_ > 700)] = 700
    x_[np.where(x_ < -700)] = -700
    #for prohibitting overflow.
    #result is not change.
    return (1 + tanh(x_))/2

X = np.arange(-5, 5, 0.01)
Y = F(X)
plt.figure(figsize=(3,2))
plt.plot(X,Y, color="black", linewidth=5)

#plt.xlim([0,10])
def Output(input_layer, middle_network, T, SeeRawVal=False):
    if SeeRawVal:
        V = (DIG_INPUT.T)[:,0]
    else:
        V = DIG_INPUT.T
    U = F(np.dot(input_layer, V))
    for i in range(len(middle_network)):
        if SeeRawVal & (i == len(middle_network)-1):
            return np.dot(middle_network[i], U)
        U = F(np.dot(middle_network[i], U))
    return U

# Non linear evaluate function
def evaluation(input_layer, middle_network, T):
    out = Output(input_layer, middle_network, T)
    #exp_out = np.exp(out)
    #sum_exp_out = np.sum(exp_out, axis=0)
    #softmax_out = exp_out/sum_exp_out
    eva = -1*np.sum((DIG_prob_arr - out)**2)
    return(eva)


In [7]:
#SET RUN TITLE
CreateGif = False
if CreateGif:
    work_dir = "/Users/itoutouma/Aokiken_jupyter/BowTieEvo/temporal_stored_data/"
    try:
        PRJ_TITLE = "Rank6_goal_fluctuationtest"
        SavePath = work_dir + "PRJ_TITLE"
        #os.chdir("/Users/itoutouma/Aokiken_jupyter/BowTieEvo/temporal_stored_data/")
        os.makedirs(f"{work_dir}PRJ_TITLE/result/")
        print(os.getcwd())
    except:
        print("File exist")
    

In [10]:
def set_func(x):
    set_result = np.zeros(6)
    if x in [2,4]: set_result[0] = 1
    if x in [2,6]: set_result[1] = 1
    if x in [2,8]: set_result[2] = 1
    if x in [4,6]: set_result[3] = 1
    if x in [4,8]: set_result[4] = 1
    if x in [6,8]: set_result[5] = 1
    return set_result
set_func(2)

array([1., 1., 1., 0., 0., 0.])

In [ ]:
nNode = 6
nMatrix = 3
nLayer = nMatrix+1

NETWORK_NORM = 0.01
DATA_SIZE=len(DIG_TRUE)
#DIG_prob_arr = np.zeros((nNode,DATA_SIZE))
DIG_prob_arr = np.ones((nNode,DATA_SIZE))
print(DIG_prob_arr.shape)
print(DATA_SIZE)
for i in range(DATA_SIZE):
    DIG_prob_arr[:, i] = set_func(DIG_TRUE[i])
print(DIG_prob_arr[0:5,0:5])



def set_eva(Ind):
    network = Ind.getGenom()
    eva = evaluation(Ind.getInputLayer(), Ind.getGenom(), Ind.getThr()) #Normal
    Ind.setEvaluation(eva)
    return(Ind)



POPULATION_SIZE = 1000
MAX_GENERATION = 500000

MUTATION_RATE = 0.2/((nMatrix*nNode*nNode) + 64*nNode)
Initialization = True
THR = np.zeros((nNode,nMatrix+1))

result_list = list()
for itr in range(1):
    print(f"itr:{itr}")
    
    if Initialization:
        current_generation_individual_group_pre = [CreateInd(nNode,nMatrix,NETWORK_NORM) for i in range(POPULATION_SIZE)]
        current_generation_individual_group = [set_eva(Ind) for Ind in  current_generation_individual_group_pre]
        meanint = np.mean(current_generation_individual_group[0].getGenom()[0])
        meanint2 = np.mean(current_generation_individual_group[0].getGenom()[1])
        meaninp = np.mean(current_generation_individual_group[0].getInputLayer())
        inout = np.dot(Total_in_out(current_generation_individual_group[0].getGenom()), current_generation_individual_group[0].getInputLayer())

        ind = current_generation_individual_group[0]
        out = Output(ind.getInputLayer(), ind.getGenom(), ind.getThr())[0:5,0:5]
        
        norm = (np.linalg.norm(inout, ord="fro")) 
        
        print(f"mean:{meanint}, norm:{norm}, out:{out}")
        ActiveNodeList = list()
        pre_count = 0
    else:
        #Update their fitness
        #DesiredGoal= CreateRandomGoalMatrix(1, 60)
        current_generation_individual_group = [set_eva(Ind) for Ind in current_generation_individual_group]
        pre_count = count
        
    #Evaluation is initialized. 
    initial_network_size = np.mean([np.linalg.norm(Total_in_out(Ind.getGenom()), 'fro') for Ind in current_generation_individual_group])
    
    end = 30100*3
    for count in range(pre_count, end):  #Repeat generation 
    
        """    
        if count%1000 == 0:
            #DesiredGoal=  Clean_Goal + np.random.normal(0, 1, (nNode, nNode)) #
            print("Goal Change")
            DesiredGoal=  CreateRandomGoalMatrix(GOAL_RANK, GOAL_NORM, variance)
            current_generation_individual_group = [set_eva(Ind) for Ind in current_generation_individual_group]
        """ 
        next_generation_network = [copy.deepcopy(Ind.getGenom()) for Ind in current_generation_individual_group]
        next_geenration_input =  [copy.deepcopy(Ind.getInputLayer()) for Ind in current_generation_individual_group]
        #next_geenration_Thr =  [copy.deepcopy(Ind.getThr()) for Ind in current_generation_individual_group]
        next_generation_individual_group = [genom(input_layer, network, 0, THR) for input_layer, network in zip(next_geenration_input, next_generation_network)]
        
        next_generation_individual_group_mutated = [add_mutation(Ind, MUTATION_RATE) for Ind in next_generation_individual_group]
        next_generation_individual_group_evaluated = [set_eva(Ind) for Ind in  next_generation_individual_group_mutated]
    
        #Selection
        mixed_population = current_generation_individual_group + next_generation_individual_group_evaluated
        #tournament size is 4
        selected_group = select(mixed_population, 4)
    
        #Generation change
        selected_inp = [copy.deepcopy(Ind.getInputLayer()) for Ind in selected_group]
        selected_net = [copy.deepcopy(Ind.getGenom()) for Ind in selected_group]
        selected_eva = [Ind.getEvaluation() for Ind in selected_group]
        #selected_Thr = [Ind.getThr() for Ind in selected_group]
        current_generation_individual_group = [genom(selected_inp[i], selected_net[i], selected_eva[i], THR) for i in range(POPULATION_SIZE)]
        
        #most_fitted = sorted(selected_group, reverse=True, key=lambda u: u.evaluation)[0]
        #ConsActiveNode = Active_node_in_MATLAB(most_fitted, mode="result")

            
        if count%100==0:
            if np.isnan(np.mean(selected_eva)):
                print("nan detected. break") 
                break
            #LossLixt.append(np.mean(selected_eva))
            #Output section for debugging
            most_fitted = sorted(selected_group, reverse=True, key=lambda u: u.evaluation)[0]
            #print("average interaction:{}".format(np.mean(most_fitted.getGenom())))
            ConsActiveNode = Active_node_in_MATLAB(most_fitted, mode="result")
            ConsActiveNode.append(np.mean(selected_eva))
            ActiveNodeList.append(ConsActiveNode)
            #print("-----第{}世代の結果-----".format(count))
            #print("Min:{}".format(min(selected_eva)))
            #print("Max:{}".format(max(selected_eva)))
            #print("Avg:{}".format(np.mean(selected_eva)))
            #print("\nGen{}, Ave Loss:{}, Max Loss:{}, Active node:\n{}, \nLabel1Pred:\n{}\n".format(count, np.mean(selected_eva), np.max(selected_eva), ConsActiveNode, out[:,1]))
            #print("Feobenius norm: {}\n".format(np.linalg.norm(Total_in_out(most_fitted.getGenom()), 'fro')))
            if CreateGif:
                #G = WriteNetwork_effect_on_layer(most_fitted, nNode, nLayer, count, False)
                tidyvisual = True
                correction_v=15
                #correction_v=2 #if it's 3 node
                G = WriteNetwork(most_fitted, nNode, nLayer, count, tidyvisual, correction_v, False)
                #T = SimpleWriteNetwork(Total_in_out(most_fitted.getGenom()), nNode, 2, count)
                Gtitle = SavePath +"/"+  str((count)) + "Gen"
                #Ttitle = SavePath + "/" + str(count) + "Tot"
                G.render(Gtitle)
                #T.render(Ttitle)
            
            inout = np.dot(Total_in_out(most_fitted.getGenom()), most_fitted.getInputLayer())
            #mfout = F(np.dot(inout, DIG_INPUT.T[:,0])
            mfout = Output(most_fitted.getInputLayer(), most_fitted.getGenom(), most_fitted.getThr())

            score_matrix = np.round(mfout) - DIG_prob_arr
            accur_score = np.sum(abs(score_matrix))/(score_matrix.shape[0]*score_matrix.shape[1])

            print("\nGen{}, Ave Loss:{}, Acurracy:{},  \nRawOutVal:{}, \nActive node:\n{}".format(count, np.mean(selected_eva), accur_score , mfout, ConsActiveNode))

        
        """
        if abs(np.mean(selected_eva)) < 0.01:
        #if abs(np.mean(selected_eva)) > 10000:
            most_fitted = sorted(selected_group, reverse=True, key=lambda u: u.evaluation)[0]
            ActiveNode = Active_node_in_MATLAB(most_fitted, mode="result")
            #Waist = np.min(ActiveNode)
            print("Min:{}".format(min(selected_eva)))
            print("Max:{}".format(max(selected_eva)))
            print("Avg:{}".format(np.mean(selected_eva)))
            print("Active node:{}".format(ActiveNode))
            print("Quit GA:{}".format(count))
            break
        """
    result_list.append(ActiveNodeList) 
    print("End")


In [13]:
Ind = most_fitted
print(Output(Ind.getInputLayer(), Ind.getGenom(), Ind.getThr()))
print(DIG_prob_arr)
oput = Output(Ind.getInputLayer(), Ind.getGenom(), Ind.getThr())

score_matrix = np.round(oput) - DIG_prob_arr
siz = score_matrix.shape[0]*score_matrix.shape[1]
accuracy_score = 1-np.sum(abs(score_matrix))/siz
print(accuracy_score )

[[0.4852 0.4853 0.4853 ... 0.4852 0.4852 0.4851]
 [0.5021 0.502  0.5022 ... 0.5019 0.5021 0.5021]
 [0.4756 0.4755 0.4756 ... 0.4755 0.4756 0.4754]
 [0.5231 0.5233 0.523  ... 0.5232 0.5231 0.5234]
 [0.491  0.4908 0.4911 ... 0.4911 0.491  0.4909]
 [0.5055 0.5055 0.5054 ... 0.5056 0.5055 0.5054]]
[[1. 1. 0. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 1.]
 [0. 1. 1. ... 1. 0. 0.]
 [0. 1. 0. ... 1. 1. 1.]
 [0. 0. 1. ... 0. 1. 1.]]
0.5025712949976624


In [222]:
#data = np.array(result_list)
#result = [np.mean(data[j], axis=0) for j in range(4)]
#np.save("NonLinear_ParamSetB_A001",ActiveNodeList)

In [ ]:
import matplotlib.pyplot as plt


N001data = list()
for i in range(10):
    N001data.append(np.load(f"./FigureData/S13/NonLinearA0_A0_001_{i}.npy"))
N001data = np.array(N001data)
print(N001data.shape)


N1000data = list()
for i in range(10):
    N1000data.append(np.load(f"./FigureData/S13/NonLinearA0_A0_1000_{i}.npy"))
N1000data = np.array(N1000data)
print(N1000data.shape)



ax = list(range(6))
fig = plt.figure(figsize=(4, 6),facecolor="white", dpi=240)

N001_colors = ["#D15235", "#455CB1", "#00AB81",  "#000000"]
N40_colors = ["#FF8B89", "#ABD0E4", "#8DBA99", "grey"]
Ranks = ["Rank1","Rank2", "Rank3", "Rank6"]
layer = 0

#Define character arrays for readability
Layers = ["Layer 1","Layer 2", "Layer 3", "Layer 4", "Layer 5", "Fitness\n(x$10^3$)"]
PannelPosition = ["left edge", 0,0,"right edge"]


active_node_list_N001 = np.mean(N001data, axis=0)[0:100]#np.array(N001_ActiveNodeList)
active_node_list_N100 = np.mean(N1000data, axis=0)[0:100]#np.array(N1000_ActiveNodeList)

#active_node_list_N100 = np.load("NonLinear_beta05_A1000_6classi_data300_nNode6.npy")
#active_node_list_N001 = np.load("NonLinear_beta05_A001_6classi_data300_nNode6.npy")
#active_node_list_N100 = np.load("NonLinear_ParamSetB_A1000.npy")
#active_node_list_N001 = np.load("NonLinear_ParamSetB_A001.npy")

col = "grey"
for i in range(6):
    
    ax[i] = fig.add_subplot(6, 1, i+1)
    if Layers[i] == "Layer 1":
        ax[i].set_ylim(0.8,2)
        ax[i].set_yticks([np.log10(1), np.log10(10),np.log10(64)])
        ax[i].set_yticklabels([1,10,64])
        ax[i].set_xticks([])
        ax[i].plot(np.log10(active_node_list_N001[:,i]), alpha= 1, color=col, label="$A_0$ 0.01")
        ax[i].plot(np.log10(active_node_list_N100[:,i]), alpha= 0.5, color=col, linestyle="dashed", label="$A_0$ 100")
        ax[i].legend(labels = ["$A_0$:0.01", "$A_0$:1000"])
        #ax[i].legend(labels = [f"$A_0$:{NETWORK_NORM}"])
    elif Layers[i] != "Fitness\n(x$10^3$)":
        ax[i].set_ylim(0,nNode+1)
        ax[i].set_yticks([1, 6])
        ax[i].set_xticks([])
        alpha_ = 1
        ax[i].plot(active_node_list_N001[:,i], alpha= 1, color=col, label="$A_0$ 0.01")
        ax[i].plot(active_node_list_N100[:,i], alpha= 0.5, color=col, linestyle="dashed", label="$A_0$ 100")
    elif Layers[i] == "Fitness\n(x$10^3$)":        
        #ax[i].set_ylim([-300,0])
        #ax[i].set_yticks([-200,0])
        #ax[i].set_yticklabels(["-2", "0"])
        alpha_ = 0.5
        ax[i].plot(active_node_list_N001[:,5], alpha= 1, color=col, label="$A_0$ 0.01")
        ax[i].plot(active_node_list_N100[:,5], alpha= 0.5, color=col, linestyle="dashed", label="$A_0$ 100")

    ax[i].set_ylabel(Layers[i],size=8)

    
    #Individual plot (first 10 sample)
    """
    
    for j in range(9):
        ax[i].plot(N001data[j][0:100][:,i], alpha=0.1, color="grey", label="$A_0$:0.01")
        ax[i].plot(N1000data[j][0:100][:,i], alpha=0.1, color="black",linestyle="dashed",label="$A_0$:100")      
    
    if Layers[layer] == "Layer 1":
        ax[i].legend(labels = ["$A_0$:0.01", "$A_0$:40"])
        ax[i].set_title(Ranks[i],size=14)

    if PannelPosition[i%4] == "left edge":
        ax[i].set_ylabel(Layers[layer],size=14)
    if PannelPosition[i%4] == "right edge":
        layer+=1
    """
    
        
#matplotlib.rcParams['pdf.fonttype'] = 42
fig.text(-0.03, 0.55, 'No. of active nodes in each layer', ha='center', va='center', rotation='vertical', size=14)
fig.text(0.5, 0.05, 'Generation (x100)', ha='center', va='center', size=14)
#fig.savefig("/Users/itoutouma/Desktop/BowTiePaper/Figures0319/Fig3set/{}.pdf".format